# Energy Demand Forecasting Analysis

This notebook demonstrates the energy demand forecasting models including Prophet, LSTM, and ensemble approaches.

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from data.data_loader import EnergyDataLoader
from features.feature_engineering import FeatureEngineer
from models.prophet_model import ProphetForecaster
from models.lstm_model import LSTMForecaster
from models.ensemble_model import EnsembleForecaster
from utils.evaluation import ModelEvaluator

## Data Loading and Exploration

In [ ]:
# Load sample data
data_loader = EnergyDataLoader()
data = data_loader.load_sample_data()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
data.head()

In [ ]:
# Plot energy demand over time
plt.figure(figsize=(15, 6))
plt.plot(data.index, data['demand'])
plt.title('Energy Demand Over Time')
plt.xlabel('Date')
plt.ylabel('Demand (MW)')
plt.grid(True, alpha=0.3)
plt.show()

## Feature Engineering

In [ ]:
# Add features
feature_engineer = FeatureEngineer()
data = feature_engineer.add_temporal_features(data)
data = feature_engineer.add_weather_features(data)
data = feature_engineer.add_lag_features(data)

print(f"Features added. New shape: {data.shape}")
print(f"Feature columns: {list(data.columns)}")

## Model Training and Evaluation

In [ ]:
# Split data
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))

train_data = data[:train_size].copy()
val_data = data[train_size:train_size + val_size].copy()
test_data = data[train_size + val_size:].copy()

print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

In [ ]:
# Train Prophet model
prophet_model = ProphetForecaster()
prophet_data = feature_engineer.prepare_prophet_data(train_data)
prophet_model.fit(prophet_data)

# Make predictions
val_prophet_data = feature_engineer.prepare_prophet_data(val_data)
prophet_predictions = prophet_model.predict(val_prophet_data)

print("Prophet model trained and predictions made")

In [ ]:
# Evaluate models
evaluator = ModelEvaluator()
prophet_metrics = evaluator.calculate_metrics(val_data['demand'].values, prophet_predictions)

print("Prophet Model Performance:")
for metric, value in prophet_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Plot predictions
fig = evaluator.plot_predictions(
    val_data['demand'].values, 
    prophet_predictions, 
    dates=val_data.index,
    title="Prophet Model Predictions"
)
plt.show()